<a href="https://colab.research.google.com/github/InsightfulSantosh/EDA-InsightHub/blob/main/VizPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import (
    PowerTransformer,
    OneHotEncoder,
    StandardScaler
)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from wordcloud import WordCloud, STOPWORDS
from IPython.display import display, HTML

In [45]:
#Adjust font size

def display_html(size=3, content="content"):
    """
    Generates and displays an HTML header of specified size and content.

    Parameters:
    - size (int): The header size (1 for <h1>, 2 for <h2>, etc.). Default is 3.
    - content (str): The text content to display within the header. Default is "content".

    Returns:
    - None: The function directly renders the HTML header in the output."""
    display(HTML(f"<h{size}>{content}</h{size}>"))


In [46]:
#Display misisng info:
def missing_info(df):
    """
    Identifies and summarizes missing data in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to analyze.

    Returns:
    - pd.DataFrame: A DataFrame with columns 'column', 'count', and 'percentage'
      indicating the column name, the number of missing values, and the percentage
      of missing values, respectively, sorted by the count of missing values in descending order.
    """
    na_col = [col for col in df.columns if df[col].isna().any()]
    missing_data = pd.DataFrame({
        "column": na_col,
        "count": df[na_col].isna().sum(),
        "percentage": df[na_col].isna().sum() / df.shape[0] * 100
    }).sort_values(by="count", ascending=False).set_index("column").reset_index()
    return missing_data

def plot_missing_info(data,rotation=45, figsize=(10, 4)):
    """
    Plots the count of missing values for each variable in a DataFrame.

    Parameters:
    - data (pd.DataFrame): The input DataFrame to analyze.
    - rotation (int): The rotation angle for x-axis labels. Default is 45 degrees.
    - figsize (tuple): The size of the figure. Default is (10, 4).

    Returns:
    - None: The function displays a bar plot showing the count of missing values per variable.
    """
    na_data = missing_info(data)
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # Create the bar plot with proper x-axis labels
    bar = ax.bar(
        range(len(na_data)),
        height=na_data["count"].values,
        color="#1eba47",
        edgecolor="black",
        tick_label=na_data['column'],  # to use column names
        alpha=0.7
    )
    # Set axis labels and title
    ax.set(
        xlabel="Variable",
        ylabel="Count",
        title="Missing Data Counts per Variable"
    )

    # Rotate x-axis labels if needed
    plt.xticks(rotation=rotation)

    # Adjust layout and display the plot
    plt.tight_layout()
    plt.show()

In [47]:
# IsolationForest for outlier detection
def outlier_detect(df):
    """
    Detects outliers in a DataFrame using the IsolationForest algorithm.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to analyze. Only numeric columns are considered for outlier detection.

    Returns:
    - pd.DataFrame: A DataFrame containing the rows identified as outliers. An additional column 'ouliers' is added with a value of -1 for outliers.
    """
    isl= IsolationForest(n_estimators=50,random_state=42)
    outlier=df.assign(ouliers=isl.fit_predict(df.iloc[:, :-1]
                .select_dtypes(include="number"))).query("ouliers==-1")
    return outlier


In [48]:
# IQR for outlier detection
def get_iqr_outliers(data, var, band=1.5):
    """
    Detects outliers in a DataFrame using the Interquartile Range (IQR) method.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing the variable to analyze.
    - var (str): The name of the column/variable for which to detect outliers.
    - band (float): The multiplier for the IQR to determine the lower and upper limits. Default is 1.5.

    Returns:
    - pd.DataFrame: A DataFrame containing the rows identified as outliers, sorted by the variable `var`.
    """
    q1, q3 = (
        data
        .loc[:, var]
        .quantile([0.25, 0.75])
        .values
    )

    iqr = q3 - q1
    lower_limit = q1 - (band * iqr)
    upper_limit = q3 + (band * iqr)

    display_html(3, f"{var} - IQR Limits:")
    print(f"{'Lower Limit':12}: {lower_limit}")
    print(f"{'Upper Limit':12}: {upper_limit}")

    outliers = data.query(f"{var} > @upper_limit | {var} < @lower_limit")
    display_html(3, f"Total outliers detected: {outliers.shape[0]}")

    return outliers.sort_values(var)

In [49]:
#pair plot
def pair_plots(data,
               height=3,
               aspect=1.5,
               hue=None,
               legend=False):
    """
    Generates pair plots (scatterplots) for the given DataFrame.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing the data to plot.
    - height (float): The height of each plot in the grid. Default is 3.
    - aspect (float): The aspect ratio of each plot (width/height). Default is 1.5.
    - hue (str or None): Variable in `data` to map plot aspects to different colors. Default is None.
    - legend (bool): Whether to add a legend to the plots. Default is False.

    Returns:
    - None: The function displays a grid of pair plots.
    """
    display_html(2, "Pair Plots")

    pair_grid = sns.PairGrid(
        data=data,
        aspect=aspect,
        height=height,
        hue=hue,
        corner=True
    )
    pair_grid.map_lower(sns.scatterplot)

    if legend:
        pair_grid.add_legend()

In [50]:
#Corelation analysis of numerical variable
def corr_heatmap(df, method="spearman",cmap='coolwarm', figsize=(8, 6)):
    """
    Generates a correlation heatmap for numerical variables in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the data for correlation analysis.
    - method (str): The correlation method to use. Options include 'pearson', 'kendall', and 'spearman'. Default is 'spearman'.
    - cmap (str): The colormap to use for the heatmap. Default is 'coolwarm'.
    - figsize (tuple): The size of the figure (width, height) in inches. Default is (8, 6).

    Returns:
    - None: The function displays a heatmap of the correlation matrix.
    """
    # Compute the correlation matrix
    cm = df.corr(method=method, numeric_only=True)

    # Create a mask for the upper triangle
    mask = np.zeros_like(cm, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=figsize)

    # Create the heatmap
    sns.heatmap(cm, annot=True, mask=mask, ax=ax, cmap=cmap, vmin=-1, vmax=1)
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    ax.set(title= f"{method.title()} Correlation Matrix Heatmap")
    # Display the plot
    plt.show()


In [51]:
##Corelation analysis of categorical variable
def cramers_heatmap(df, cmap='coolwarm', figsize=(8, 6)):
    """
    Generates a Cramér's V correlation heatmap for categorical variables in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the categorical data for correlation analysis.
    - cmap (str): The colormap to use for the heatmap. Default is 'coolwarm'.
    - figsize (tuple): The size of the figure (width, height) in inches. Default is (8, 6).

    Returns:
    - None: The function displays a heatmap of the Cramér's V correlation matrix.

    Example:
    cramers_heatmap(df, cmap="viridis", figsize=(10, 8))
    """
    # Get categorical columns
    cat_cols = df.select_dtypes(include='object').columns
    # Initialize a DataFrame for the correlation matrix
    cramers_v_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)

    # Compute Cramér's V for each pair of categorical columns
    for i in cat_cols:
        for j in cat_cols:
            if i == j:
                cramers_v_matrix.loc[i, j] = 1.0  # Diagonal should be 1.0
            else:
                cramers_v_matrix.loc[i, j] = cramers_v(df, i, j)

    # Convert the Cramér's V matrix to numeric type
    cramers_v_matrix = cramers_v_matrix.astype(float)

    # Create a mask for the upper triangle
    mask = np.zeros_like(cramers_v_matrix, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    # Plot heatmap
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(
        cramers_v_matrix,
        vmin=0,
        vmax=1,
        cmap=cmap,
        annot=True,
        fmt=".2f",
        square=True,
        linewidths=1.5,
        mask=mask,
        ax=ax
    )
    ax.set(title="Cramer's V Correlation Matrix Heatmap")
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [52]:
# Summary of categorical variables:
def cat_summary(df, var):
    """
    Generates a summary report for a categorical variable in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the categorical data.
    - var (str): The name of the categorical variable to summarize.

    Returns:
    - None: The function displays various statistics and information about the specified categorical variable, including:
        - A quick glance at the data in the variable.
        - Metadata such as data type, cardinality (number of unique values), and missing data information.
        - Summary statistics.
        - Distribution of categories in terms of count and percentage.
    """
    # Extract the column specified by 'var' from the DataFrame 'df'
    col = df.loc[:, var].copy()

    # Display the variable name as a header
    display_html(3, var)

    # Display a quick glance at the column
    display_html(3, "Quick Glance:")
    print(col)

    # Display metadata about the column
    display_html(3, "Meta-data:")
    print(f"Datatype:       {col.dtype}")  # Data type of the column
    print(f"Cardinality:    {col.nunique(dropna=True)}")  # Number of unique values (excluding NaN)
    print(f"Missing data:   {col.isna().sum()} rows ({col.isna().sum()/df.shape[0]:.2%})")  # Missing values count and percentage
    print(f"Available data: {(~col.isna()).sum()} / {col.shape[0]}")  # Available data count

    # Display summary statistics of the column
    display_html(3, "Summary:")
    display(col.describe().rename("").to_frame())  # Summary statistics for numeric columns

    # Display category distribution
    display_html(3, "Categories Distribution:")
    with pd.option_context("display.max_rows", None):
        # Create a DataFrame with count and percentage of each category
        display(pd.DataFrame({"count": col.value_counts(),
                              "percentage": (col.value_counts() / col.shape[0])*100})
                .rename_axis(index="category"))


In [53]:
# Summary of Numerical variables:

def Num_summary(df, var):
    """
    Generates a detailed summary report for a numerical variable in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the numerical data.
    - var (str): The name of the numerical variable to summarize.

    Returns:
    - None: The function displays various statistics and information about the specified numerical variable, including:
        - A quick glance at the data in the variable.
        - Metadata such as data type, missing data information, and available data count.
        - Percentiles, including the 0th, 5th, 10th, 25th, 50th (median), 75th, 90th, 95th, 99th, and 100th percentiles.
        - Central tendency measures like mean, trimmed mean (5% and 10%), and median.
        - Measures of spread such as variance, standard deviation, interquartile range (IQR), median absolute deviation (MAD),
        and coefficient of variation.
        - Skewness and kurtosis.
        - Hypothesis testing for normality."""
    # Extract the column specified by 'var' from the DataFrame 'df'
    col = df.loc[:, var].copy()

    # Display the variable name as a header
    display_html(3, var)

    # Display a quick glance at the column
    display_html(3, "Quick Glance:")
    print(col)

    # Display metadata about the column
    display_html(3, "Meta-data:")
    print(f"Datatype:       {col.dtype}")  # Data type of the column
    print(f"Missing data:   {col.isna().sum()} rows ({col.isna().sum()/df.shape[0]:.2%})")  # Missing values count and percentage
    print(f"Available data: {(~col.isna()).sum()} / {col.shape[0]}")  # Available data count

    #Percentile
    display_html(3, "Percentiles:")
    display(col
        .quantile([0.0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1.0])
        .reset_index().rename(columns={"index":"percentile",var:"value"})
        .assign(percentile=(lambda x: x["percentile"]*100)) # Multiply only the 'percentile' column by 100
        .set_index('percentile')
        )
    # central tendancy
    display_html(3, "Central Tendancy:")
    display(
        pd
        .Series({"mean": col.mean(),
                "trimmed mean (5%)": stats.trim_mean(col.values, 0.05),
                "trimmed mean (10%)": stats.trim_mean(col.values, 0.1),
                "median": col.median()})
        .rename("value")
        .to_frame()
        )
    # spread
    display_html(3, "Measure of Spread:")
    std = col.std()
    iqr = col.quantile(0.75) - col.quantile(0.25)
    display(
        pd
        .Series({
            "var": col.var(),
            "std": std,
            "IQR": iqr,
            "mad": stats.median_abs_deviation(col.dropna()),
            "coef_variance": std / col.mean()
        })
        .rename("value")
        .to_frame()
    )

    # skewness and kurtosis
    display_html(3, "Skewness and Kurtosis:")
    display(
        pd
        .Series({
            "skewness": col.skew(),
            "kurtosis": col.kurtosis()
        })
        .rename("value")
        .to_frame()
    )
    alpha = 0.05
    # test for normality
    display_html(3, "Hypothesis Testing for Normality:")
    # shapiro-wilk test
    display_html(4, "Shapiro-Wilk Test:")
    sw_test = stats.shapiro(col.dropna().sample(4500).values)
    sw_statistic = sw_test.statistic
    sw_pvalue = sw_test.pvalue
    print(f"{'Significance Level':21}: {alpha}")
    print(f"{'Null Hypothesis':21}: The data is normally distributed")
    print(f"{'Alternate Hypothesis':21}: The data is not normally distributed")
    print(f"{'p-value':21}: {sw_pvalue}")
    print(f"{'Test Statistic':21}: {sw_statistic}")
    if sw_pvalue < alpha:
        print(f"- Since p-value is less than alpha ({alpha}), we Reject the Null Hypothesis at {alpha * 100}% significance level")
        print("- CONCLUSION: We conclude that the data sample is not normally distributed")
    else:
        print(f"- Since p-value is greater than alpha ({alpha}), we Fail to Reject the Null Hypothesis at {alpha * 100}% significance level")
        print("- CONCLUSION: We conclude that the data sample is normally distributed")

    #anderson-darling test
    display_html(4, "Anderson-Darling Test:")
    ad_test = stats.anderson(col.dropna().values, dist="norm")
    ad_statistic = ad_test.statistic
    ad_critical = ad_test.critical_values[2]
    print(f"{'Significance Level':21}: {alpha}")
    print(f"{'Null Hypothesis':21}: The data is normally distributed")
    print(f"{'Alternate Hypothesis':21}: The data is not normally distributed")
    print(f"{'Critical Value':21}: {ad_critical}")
    print(f"{'Test Statistic':21}: {ad_statistic}")
    if ad_statistic >= ad_critical:
        print(f"- Since the Test-statistic is greater than Critical Value, we Reject the Null Hypothesis at {alpha * 100}% significance level")
        print("- CONCLUSION: We conclude that the data sample is not normally distributed")
    else:
        print(f"- Since the Test-statistic is less than Critical Value, we Fail to Reject the Null Hypothesis at {alpha * 100}% significance level")
        print("- CONCLUSION: We conclude that the data sample is normally distributed")


In [54]:
# Plot categorical variable (univariate analysis)
def cat_univar_plots(data, var, k=None, show_wordcloud=True, figsize=(12, 8.5)):
    """
    Generates univariate visualizations for a categorical variable in a DataFrame.

    Parameters:
    - data (pd.DataFrame): The DataFrame containing the categorical data.
    - var (str): The name of the categorical variable to visualize.
    - k (int, optional): If specified, only the top k categories by count are included. The rest are grouped into "Other".
    - show_wordcloud (bool, optional): Whether to include a word cloud visualization. Default is True.
    - figsize (tuple, optional): The size of the figure for the plots. Default is (12, 8.5).

    Returns:
    - None: Displays a series of plots including:
        - A bar chart showing counts of categories.
        - A pie chart showing the percentage distribution of categories.
        - (Optional) A word cloud visualization of the categories.
    """
    display_html(1,f"Univariate Analysis of {var}")
    # Data Processing
    col = data[var].copy()
    if k is not None:
        top_categories = col.value_counts().index[:k-1]
        counts = col.where(col.isin(top_categories), "Other").value_counts()
    else:
        counts = col.value_counts()

    # Random Colors
    colors = [tuple(np.random.choice(256, size=3) / 255) for _ in range(len(counts))]

    # Plotting
    fig = plt.figure(figsize=figsize)
    gs = GridSpec(2, 2, figure=fig)
    ax1, ax2, ax3 = fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[1, :])

    # Bar-chart
    ax1.bar(counts.index, counts.values, color=colors, edgecolor="black", alpha=0.7)
    ax1.bar_label(ax1.containers[0], padding=5, color="black")
    ax1.set(title="Bar Chart", xlabel="Categories", ylabel="Count")
    ax1.tick_params(axis='x', rotation=45)

    # Pie-chart
    ax2.pie(counts, labels=counts.index, autopct="%.2f%%", colors=colors, wedgeprops=dict(alpha=0.7, edgecolor="black"))
    ax2.set_title("Pie Chart")
    ax2.legend(loc="upper left", bbox_to_anchor=(1.02, 1), title="Categories")

    # Word-cloud
    if show_wordcloud:
        var_string = " ".join(
            data
            .loc[:, var]
            .dropna()
            .str.replace(" ", ".")
            .to_list()
        )

        word_cloud = WordCloud(
            width=2000,
            height=700,
            random_state=42,
            background_color="black",
            colormap="Set2",
            stopwords=STOPWORDS
        ).generate(var_string)

        ax3.imshow(word_cloud, interpolation="bilinear")
        ax3.axis("off")
        ax3.set_title("Word Cloud")
    else:
        ax3.remove()

    plt.tight_layout()
    plt.show()

In [55]:
# Plot categorical-numerical variable (Bivariate analysis)

def num_cat_bivar_plots(df, var, num, k=None, estimator="mean",rotation=90, figsize=(12,10)):

    """
    Generate a set of bi-variate plots between a categorical variable and a numerical variable.

    This function creates three plots to visualize the relationship between a categorical variable
    (`var`) and a numerical variable (`num`):
    1. A bar plot showing the aggregated values (mean or other estimator) of the numerical variable
       for each category.
    2. A boxplot illustrating the distribution of the numerical variable across the categories.
    3. A violin plot providing a density estimate of the numerical variable for each category.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        var (str): The name of the categorical variable.
        num (str): The name of the numerical variable.
        k (int, optional): The number of top categories to include. Categories beyond this number
            will be grouped into an "Other" category. If `None`, all categories are used.
        estimator (str, optional): The aggregation function to apply to the numerical variable
            (e.g., "mean", "sum"). Default is "mean".
        figsize (tuple, optional): The size of the figure. Default is (12, 10).

    """

    display_html(1, f"Bi-variate Analysis between {var} and {num}")

    # Make a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Handle top k categories if specified
    if k is not None:
        top_categories = df_copy[var].value_counts().index[:k-1]
        df_copy[var] = df_copy[var].where(df_copy[var].isin(top_categories), "Other")  # Create "Other" category

    # Calculate aggregated values for barplot
    agg_df = df_copy.groupby(var)[num].agg(estimator).reset_index().sort_values(by=num, ascending=True)

    # Order categories by aggregated values
    order = agg_df[var]

    # Plotting
    fig = plt.figure(figsize=figsize)
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, :])

    # Create the barplot
    sns.barplot(data=agg_df, x=var, y=num, ax=ax1, color="#d92b2b", order=order)
    ax1.bar_label(ax1.containers[0], padding=2, color="black")  # Set labels to vertical
    ax1.set_title(f'{estimator.capitalize()} {num} by {var}')
    ax1.tick_params(axis='x', rotation=rotation)

    # Create the boxplot
    sns.boxplot(data=df_copy, x=var, y=num, order=order, color="lightgreen", ax=ax2)
    ax2.set_title(f'Boxplot of {num} by {var}')
    ax2.tick_params(axis='x', rotation=rotation)

    # Create the violinplot
    sns.violinplot(data=df_copy, x=var, y=num, order=order, ax=ax3)
    ax3.set_title(f'Violinplot of {num} by {var}')
    ax3.tick_params(axis='x', rotation=rotation)

    plt.tight_layout()
    plt.show()

In [56]:
# Cat-Cat plot (Bivariate plot)
def plot_cat_bivariate(data, var1, var2, k1=None, k2=None, order1=None, order2=None, figsize=(15, 15), xlabel_rotation=45, annot_fontsize=10):
    """
    Function to perform bivariate analysis between two categorical variables,
    generating a cross-tab heatmap, normalized heatmap, bar plot, and stacked bar plot.

    Parameters:
    data (pd.DataFrame): DataFrame containing the data
    var1 (str): First categorical variable
    var2 (str): Second categorical variable
    k1 (int, optional): Top k categories to consider for var1
    k2 (int, optional): Top k categories to consider for var2
    order1 (list, optional): Custom order for categories of var1
    order2 (list, optional): Custom order for categories of var2
    figsize (tuple, optional): Figure size for the plots
    xlabel_rotation (int, optional): Degree of rotation for x-axis labels in the bar plots
    annot_fontsize (int, optional): Font size for annotations in the heatmap

    Returns:
    None
    """
    # Function to get top k categories
    def get_top_k(data, var, k):
        return data[data[var].isin(data[var].value_counts().nlargest(k).index)]

    # Filter top k categories if specified
    if k1 is not None:
        data = get_top_k(data, var1, k1)
    if k2 is not None:
        data = get_top_k(data, var2, k2)

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    axes = axes.ravel()

    # Cross-tab heatmap
    ct = pd.crosstab(index=data[var1], columns=data[var2]).reindex(index=order1, columns=order2)
    sns.heatmap(ct, annot=True, annot_kws={"size": annot_fontsize}, linewidths=2, linecolor="white", square=True, cmap="Blues", cbar_kws=dict(location="right", label="Counts"), ax=axes[0])
    axes[0].set_title('Cross-tab Heatmap', fontsize=14)
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=xlabel_rotation, ha="right")
    axes[0].set_xlabel(var2, fontsize=12)
    axes[0].set_ylabel(var1, fontsize=12)

    # Normalized cross-tab heatmap
    norm_ct = pd.crosstab(index=data[var1], columns=data[var2], normalize="index").reindex(index=order1, columns=order2)
    sns.heatmap(norm_ct, annot=True, annot_kws={"size": annot_fontsize}, linewidths=2, linecolor="white", square=True, cmap="Greens", cbar_kws=dict(location="right", label="Normalized"), ax=axes[1])
    axes[1].set_title('Normalized Cross-tab Heatmap', fontsize=14)
    axes[1].set_xlabel(var2, fontsize=12)
    axes[1].set_ylabel(var1, fontsize=12)
    axes[1].set(ylabel="")

    # Bar plot
    ct.plot.bar(ax=axes[2], title="Bar Plot", legend=False)
    axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=xlabel_rotation, ha="right")
    axes[2].set_xlabel(var1, fontsize=12)
    axes[2].set_ylabel('Counts', fontsize=12)

    # Stacked bar plot
    norm_ct.plot.bar(stacked=True, ax=axes[3], title="Stacked Bar Plot")
    axes[3].set_xticklabels(axes[3].get_xticklabels(), rotation=xlabel_rotation, ha="right")
    axes[3].legend(loc="upper left", bbox_to_anchor=(1, 1), title=var2)
    axes[3].set_xlabel(var1, fontsize=12)
    axes[3].set_ylabel('Proportion', fontsize=12)

    plt.tight_layout(pad=3.0)
    plt.show()



In [57]:
# Plot categorical-numerical variable (multivariate analysis)

def cat_num_facetgrid(
    data_frame, facet_column, hue_column, x_axis_column, y_axis_column, top_k=None,
    columns_per_row=None, facet_height=4, facet_aspect_ratio=1.2, x_axis_order=None
):
    """
    Creates a FacetGrid bar plot with optional top-k filtering for categories.

    Parameters:
    - data_frame: DataFrame containing the data.
    - facet_column: Column name for facetting (grid columns).
    - hue_column: Column name for hue (color).
    - x_axis_column: Column name for x-axis.
    - y_axis_column: Column name for y-axis.
    - top_k: Optional integer specifying the number of top categories to display.
    - columns_per_row: Number of facets to wrap in a row. If None, it will be auto-calculated.
    - facet_height: Height of each facet.
    - facet_aspect_ratio: Aspect ratio of each facet.
    - x_axis_order: Optional list of order for x-axis categories.
    """

    # Display a message about the analysis
    display_html(1, f"Multivariate Analysis between {x_axis_column}, {y_axis_column}, {facet_column} & {hue_column}:")

    # Copy the DataFrame to avoid modifying the original
    df_copy = data_frame.copy()

    # Handle top k categories for x_axis_column if specified
    if top_k is not None:
        top_categories = df_copy[x_axis_column].value_counts().index[:top_k]
        df_copy[x_axis_column] = df_copy[x_axis_column].where(df_copy[x_axis_column].isin(top_categories), "Other")

    # Determine the number of facets and auto-calculate columns_per_row if not provided
    num_facets = df_copy[facet_column].nunique()
    if columns_per_row is None:
        columns_per_row = max(3, num_facets // 2)  # Ensure at least 2 columns per row

    # Create the FacetGrid
    grid = sns.FacetGrid(
        data=df_copy,
        col=facet_column,
        hue=hue_column,
        col_wrap=columns_per_row,
        height=facet_height,
        aspect=facet_aspect_ratio,
        sharey=True
    )

    # Map the barplot onto the grid
    grid.map(sns.barplot, x_axis_column, y_axis_column, order=x_axis_order, errorbar=None)

    # Customize each subplot
    for ax in grid.axes.flatten():
        ax.legend(title=hue_column, bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside
        plt.setp(ax.get_xticklabels(), rotation=90)  # Rotate x-tick labels

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()

In [58]:
#Cat-Num hypothesis testing
def num_cat_hyp_testing(df,num_var,cat_var,alpha=.05):
    """
    Performs hypothesis testing to assess the association between a numerical variable and a categorical variable.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the variables.
    - num_var (str): The name of the numerical variable.
    - cat_var (str): The name of the categorical variable.
    - alpha (float, optional): The significance level for the hypothesis tests. Default is 0.05.

    Returns:
    - None: Displays the results of ANOVA and Kruskal-Wallis tests.
    """
    grouped_df=[j[num_var] for i,j in df.groupby(cat_var)]
    #ANOVA
    statistic,pvalue=f_oneway(*grouped_df)

    display_html(1,f"Hypothesis Test for Association between {num_var} and {cat_var}")
    display_html(1,"ANOVA Test")
    print(f"- {'Significance Level':21}: {alpha * 100}%")
    print(f"- {'Null Hypothesis':21}: The groups have similar population mean")
    print(f"- {'Alternate Hypothesis':21}: The groups don't have similar population mean")
    print(f"- {'Test Statistic':21}: {statistic}")
    print(f"- {'p-value':21}: {pvalue}")
    if pvalue < alpha:
        print(f"- Since p-value is less than {alpha}, we Reject the Null Hypothesis at {alpha * 100}% significance level")
        print(f"- CONCLUSION: The variables {num_var} and {cat_var} are associated to each other")
    else:
        print(f"- Since p-value is greater than {alpha}, we Fail to Reject the Null Hypothesis at {alpha * 100}% significance level")
        print(f"- CONCLUSION: The variables {num_var} and {cat_var} are not associated to each other")
    # kruskal-wallis test
    kr = kruskal(*grouped_df)
    statistic = kr[0]
    pvalue = kr[1]
    display_html(3, "Kruskal-Wallis Test")
    print(f"- {'Significance Level':21}: {alpha * 100}%")
    print(f"- {'Null Hypothesis':21}: The groups have similar population median")
    print(f"- {'Alternate Hypothesis':21}: The groups don't have similar population median")
    print(f"- {'Test Statistic':21}: {statistic}")
    print(f"- {'p-value':21}: {pvalue}")
    if pvalue < alpha:
        print(f"- Since p-value is less than {alpha}, we Reject the Null Hypothesis at {alpha * 100}% significance level")
        print(f"- CONCLUSION: The variables {num_var} and {cat_var} are associated to each other")
    else:
        print(f"- Since p-value is greater than {alpha}, we Fail to Reject the Null Hypothesis at {alpha * 100}% significance level")
        print(f"- CONCLUSION: The variables {num_var} and {cat_var} are not associated to each other")


In [59]:
#Cat-Cat hypothesis testing
def hyp_cat_cat(data, var1, var2, alpha=0.05):
    """
    Perform a hypothesis test for association between two categorical variables using the Chi-square test
    and calculate Cramér's V statistic.

    Parameters:
    data (pd.DataFrame): DataFrame containing the data.
    var1 (str): First categorical variable.
    var2 (str): Second categorical variable.
    alpha (float, optional): Significance level for the test. Default is 0.05.

    Returns:
    None
    """
    display_html(2, f"Hypothesis Test for Association between {var1} and {var2}")

    # Cross-tabulation between the two variables
    ct = pd.crosstab(data[var1], data[var2])

    # Performing the Chi-square test
    chi2_statistic, pvalue, _, _ = stats.chi2_contingency(ct)

    # Calculating Cramér's V
    n = ct.sum().sum()
    r, k = ct.shape
    cramers_v = np.sqrt(chi2_statistic / (n * (min(r, k) - 1)))

    # Displaying test results
    display_html(3,"Chi-square Test Results")
    print(f"- {'Cramérs V':21}: {cramers_v:.4f}")
    print(f"- {'Significance Level':21}: {alpha * 100:.2f}%")
    print(f"- {'Null Hypothesis':21}: The variables are independent")
    print(f"- {'Alternate Hypothesis':21}: The variables are dependent")
    print(f"- {'Chi-square Statistic':21}: {chi2_statistic:.4f}")
    print(f"- {'p-value':21}: {pvalue:.4f}")

    # Hypothesis conclusion
    if pvalue < alpha:
        print(f"\nConclusion: Since the p-value is less than {alpha}, we reject the null hypothesis.")
        print(f"The variables {var1} and {var2} are likely associated.")
    else:
        print(f"\nConclusion: Since the p-value is greater than {alpha}, we fail to reject the null hypothesis.")
        print(f"The variables {var1} and {var2} are likely independent.")


In [60]:
#Date_Time analysis:
def dt_univariate_plot(df, var, parm=None, bin="auto",fig=(12, 4)):
    """
    Performs univariate analysis of a datetime variable and visualizes its distribution.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the datetime variable.
    - var (str): The name of the datetime variable.
    - parm (str, optional): A string specifying the datetime attribute to extract. E.g., 'dt.year', 'dt.month'.
    - bin (int or 'auto', optional): Number of bins for the histogram. Default is 'auto'.
    - fig (tuple, optional): Size of the figure. Default is (12, 4).

    Returns:
    - None: Displays the histogram with a rug plot and a line plot.
    """
    # Extract datetime attribute if parm is provided
    if parm:
        if parm == 'dt.isocalendar().week':
            extracted_data = df[var].dt.isocalendar().week
        else:
            accessor, attribute = parm.split(".")
            if accessor != 'dt':
                raise ValueError("Invalid accessor. Only 'dt' accessor is supported.")
            if attribute.endswith("()"):
                attribute = attribute[:-2]  # Remove '()'
                extracted_data = getattr(df[var].dt, attribute)()
            else:
                extracted_data = getattr(df[var].dt, attribute)
    else:
        extracted_data = df[var]

    # Create figure and subplots
    fig, ax = plt.subplots(1, 2, figsize=fig)

    # Histogram with rug plot
    sns.histplot(extracted_data, kde=False, color='skyblue', bins=bin, ax=ax[0])
    sns.rugplot(x=extracted_data, color='black', ax=ax[0])
    ax[0].set_xlabel(var)
    ax[0].set_ylabel('Count')
    ax[0].set_title('Histogram with Rug Plot')
    ax[0].tick_params(axis='x', rotation=45)

    ax[0].bar_label(ax[0].containers[0], padding=3, color="black")

    # Line plot
    journey_counts = extracted_data.value_counts().sort_index()

    sns.lineplot(x=journey_counts.index, y=journey_counts.values, color="#d92b2b", ax=ax[1])
    ax[1].set_xlabel(var)
    ax[1].set_ylabel('Frequency')
    ax[1].set_title('Line Plot')
    ax[1].tick_params(axis='x', rotation=45)

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [61]:

def assign_time_of_day(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a new column to the DataFrame that categorizes the departure times into different parts of the day.

    The new column `dep_time_of_day` is created based on the hour of the day from the `dep_time` column.
    The times are categorized as follows:
    - "morning": 04:00 to 12:00 (inclusive)
    - "afternoon": 12:01 to 16:00 (inclusive)
    - "evening": 16:01 to 20:00 (inclusive)
    - "night": 20:01 to 03:59 (default for times not covered by the above ranges)

    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing a datetime column `dep_time`.

    Returns:
    -------
    pd.DataFrame
        The modified DataFrame with the new column `dep_time_of_day`.
    """
    df = df.assign(
        dep_time_of_day=lambda df_: np.select(
            [df_.dep_time.dt.hour.between(4, 12, inclusive="left"),
             df_.dep_time.dt.hour.between(12, 16, inclusive="left"),
             df_.dep_time.dt.hour.between(16, 20, inclusive="left")],
            ["morning", "afternoon", "evening"],
            default="night"
        )
    )
    return df


In [62]:
def dt_multivatirate_plot(data,dt_var,cat_var,num_var):
    """
    Creates a bar plot to visualize the mean of a numerical variable across different months
    for each category.

    Parameters:
    - data (pd.DataFrame): The DataFrame containing the variables.
    - dt_var (str): The name of the datetime variable.
    - cat_var (str): The name of the categorical variable.
    - num_var (str): The name of the numerical variable.

    Returns:
    - None: Displays a bar plot.
    """
    data=df.groupby([dt_var, cat_var]).agg({
        # Changed num_var to price as destination is a categorical variable not numerical
        num_var: 'mean',
        dt_var: lambda x: x.dt.month_name().iloc[0]
    }).reset_index(names=['date', cat_var, num_var, 'month']).iloc[:,1:]

    sns.barplot(data, x=dt_var, y=num_var, hue=cat_var,errorbar=None) # Changed num_var to price
    plt.xticks(rotation=45)
    plt.show()


In [63]:
def dt_multivariate_plot(df, dt_var, cat_var, num_var, parm=None, fig=(11.5, 6),rotate=45):
    """
    Creates a bar plot to visualize the mean of a numerical variable across different datetime
    attributes and categories.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the variables.
    - dt_var (str): The name of the datetime variable.
    - cat_var (str): The name of the categorical variable.
    - num_var (str): The name of the numerical variable.
    - parm (str, optional): A datetime accessor attribute. Default is None.
    - fig (tuple, optional): Size of the figure. Default is (11.5, 6).
    - rotate (int, optional): Rotation angle for x-axis labels. Default is 45.

    Returns:
    - None: Displays a bar plot.
    """
    df=df.copy()
    # Display a message about the analysis
    display_html(1, f"Multivariate Analysis between {dt_var}, {cat_var} & {num_var}:")
    # Handle datetime accessor if provided in parm
    if parm:
        if parm == 'dt.isocalendar().week':
            df['processed_dt_var'] = df[dt_var].dt.isocalendar().week
        else:
            accessor, attribute = parm.split(".")
            if accessor != 'dt':
                raise ValueError("Invalid accessor. Only 'dt' accessor is supported.")
            if attribute.endswith("()"):
                attribute = attribute[:-2]  # Remove '()'
                df['processed_dt_var'] = getattr(df[dt_var].dt, attribute)()
            else:
                df['processed_dt_var'] = getattr(df[dt_var].dt, attribute)
    else:
        df['processed_dt_var'] = df[dt_var]

    # Group data by processed datetime and categorical variable
    grouped_data = df.groupby(['processed_dt_var', cat_var]).agg({num_var: 'mean'}).reset_index()

    # Plot the bar plot
    plt.figure(figsize=fig)
    sns.barplot(data=grouped_data, x='processed_dt_var', y=num_var, hue=cat_var, errorbar=None)
    plt.xlabel(dt_var if parm is None else f'{dt_var} ({parm})')
    plt.ylabel(f'Mean {num_var}')
    plt.title(f'Mean {num_var} by {cat_var} and {dt_var}')

    # Adjust legend and layout
    plt.gca().legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.xticks(rotation=rotate)
    plt.tight_layout(rect=[0, 0, 1.2, 1.1])

    # Show the plot
    plt.show()

In [64]:
# Plot Numerical variable (univariate analysis)
def num_univar_plots(df, column_name, bin="auto", fig_size=(12, 10)):
    """
    Creates various univariate plots for analyzing the distribution of a numerical variable.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - column_name (str): The name of the numerical column to analyze.
    - bin (int or 'auto', optional): Number of bins for the histogram. Default is 'auto'.
    - fig_size (tuple, optional): Size of the figure. Default is (12, 10).

    Returns:
    - None: Displays the plots.
    """

    # Display a message about the analysis
    display_html(1,f"Distribution of {column_name}:")
    print()
    # Setup the figure and GridSpec
    fig = plt.figure(figsize=fig_size)
    gs = GridSpec(3, 2, height_ratios=[2, 2, 2])

    # Histogram with KDE & Rug plot
    ax0 = fig.add_subplot(gs[0, 0])
    sns.histplot(df[column_name], kde=True, color="orange", bins=bin, ax=ax0)
    sns.rugplot(df[column_name],color="black",ax=ax0)
    ax0.set_title("Histogram with KDE & Rug plot")
    ax0.set_xlabel(column_name)
    ax0.set_ylabel("Frequency")

    # ECDF Plot
    ax1 = fig.add_subplot(gs[0, 1])
    sns.ecdfplot(df[column_name], color="red", ax=ax1)
    ax1.set_title("ECDF Plot")
    ax1.set_xlabel(column_name)
    ax1.set_ylabel("ECDF")

    # Box Plot
    ax2 = fig.add_subplot(gs[1, 0])
    sns.boxplot(x=df[column_name], color="green", ax=ax2)
    ax2.set_title("Box Plot")
    ax2.set_xlabel(column_name)

    # Violin Plot
    ax3 = fig.add_subplot(gs[1, 1])
    sns.violinplot(x=df[column_name], color="purple", ax=ax3)
    ax3.set_title("Violin Plot")
    ax3.set_xlabel(column_name)

    # Q-Q Plot
    ax4 = fig.add_subplot(gs[2, :])
    stats.probplot(df[column_name], dist="norm", plot=ax4)
    ax4.set_title("Q-Q Plot")

    # Adjust layout
    plt.tight_layout()
    plt.show()


In [65]:
#Transformation Plot for numerical variable
def transformation_plot(df, column_name):
    """
    Creates a set of plots to analyze the effects of different data transformations on a numerical variable.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - column_name (str): The name of the numerical column to analyze.

    Returns:
    - None: Displays the plots.
    """

    # Define a function to create a Q-Q plot
    def qq_plot(data, ax, title):
        """Creates a Q-Q plot."""
        stats.probplot(data, dist="norm", plot=ax)
        ax.get_lines()[1].set_color('red')  # Change color of the Q-Q line
        ax.set_title(title, fontsize=16)
        ax.get_xaxis().set_tick_params(labelsize=12)
        ax.get_yaxis().set_tick_params(labelsize=12)

    # Display a message about the analysis
    display_html(1, f"Transformation Plot for {column_name}:")

    df = df.copy()

    # Creating transformed columns
    df['log_' + column_name] = np.log(df[column_name] + 0.001)
    df['sqrt_' + column_name] = np.sqrt(df[column_name] + 0.001)

    # Yeo-Johnson Transformation
    pt_yj = PowerTransformer(method='yeo-johnson')
    df['yeojohnson_' + column_name] = pt_yj.fit_transform(df[column_name].values.reshape(-1, 1))

    # Box-Cox Transformation with three conditions
    boxcox_transformed = False
    if (df[column_name] > 0).all():
        # Positive values
        pt_bc = PowerTransformer(method='box-cox')
        df['boxcox_' + column_name] = pt_bc.fit_transform(df[column_name].values.reshape(-1, 1))
        boxcox_transformed = True
    elif (df[column_name] == 0).any():
        # Zero values
        df['boxcox_' + column_name] = pt_bc.fit_transform((df[column_name]+0.001).values.reshape(-1, 1))
        boxcox_transformed = True
    else:
        # Negative values
        df['boxcox_' + column_name] = np.nan  # Set to NaN if not applicable

    # Plotting
    fig, axes = plt.subplots(5, 2, figsize=(18, 24))  # Layout for side-by-side plots

    # Original Data
    sns.histplot(df[column_name], kde=True, color="blue", ax=axes[0, 0])
    sns.rugplot(x=df[column_name], color="black", ax=axes[0, 0])
    axes[0, 0].set_title("Original Data - Histogram", fontsize=16)
    qq_plot(df[column_name], axes[0, 1], "Q-Q Plot - Original Data")

    # Log Transformation
    sns.histplot(df['log_' + column_name], kde=True, color="green", ax=axes[1, 0])
    sns.rugplot(x=df['log_' + column_name], color="black", ax=axes[1, 0])
    axes[1, 0].set_title("Log Transformation - Histogram", fontsize=16)
    qq_plot(df['log_' + column_name], axes[1, 1], "Q-Q Plot - Log Transformation")

    # Square Root Transformation
    sns.histplot(df['sqrt_' + column_name], kde=True, color="red", ax=axes[2, 0])
    sns.rugplot(x=df['sqrt_' + column_name], color="black", ax=axes[2, 0])
    axes[2, 0].set_title("Square Root Transformation - Histogram", fontsize=16)
    qq_plot(df['sqrt_' + column_name], axes[2, 1], "Q-Q Plot - Square Root Transformation")

    # Box-Cox Transformation
    if boxcox_transformed:
        sns.histplot(df['boxcox_' + column_name], kde=True, color="orange", ax=axes[3, 0])
        sns.rugplot(x=df['boxcox_' + column_name], color="black", ax=axes[3, 0])
        axes[3, 0].set_title("Box-Cox Transformation - Histogram", fontsize=16)
        qq_plot(df['boxcox_' + column_name], axes[3, 1], "Q-Q Plot - Box-Cox Transformation")
    else:
        # Remove the Box-Cox subplot if transformation is not applicable
        axes[3, 0].remove()
        axes[3, 1].remove()

    # Yeo-Johnson Transformation
    sns.histplot(df['yeojohnson_' + column_name], kde=True, color="purple", ax=axes[4, 0])
    sns.rugplot(x=df['yeojohnson_' + column_name], color="black", ax=axes[4, 0])
    axes[4, 0].set_title("Yeo-Johnson Transformation - Histogram", fontsize=16)
    qq_plot(df['yeojohnson_' + column_name], axes[4, 1], "Q-Q Plot - Yeo-Johnson Transformation")

    # Adjust subplot spacing
    plt.subplots_adjust(wspace=0.3, hspace=0.4)
    plt.tight_layout()

    plt.show()



In [66]:
# bivariate plots between 2 numeric variables
def num_bivar_plots(data, var_x, var_y, figsize=(12, 4.5), scatter_kwargs=dict(), hexbin_kwargs=dict()):
    """
    Creates bivariate plots between two numeric variables.

    Parameters:
    - data (pd.DataFrame): The DataFrame containing the numeric variables.
    - var_x (str): The name of the variable to be plotted on the x-axis.
    - var_y (str): The name of the variable to be plotted on the y-axis.
    - figsize (tuple, optional): The size of the figure. Defaults to (12, 4.5).
    - scatter_kwargs (dict, optional): Additional keyword arguments to pass to the scatter plot.
    - hexbin_kwargs (dict, optional): Additional keyword arguments to pass to the hexbin plot.

    Returns:
    - None: Displays the scatter and hexbin plots.
    """
    display_html(2, f"Bi-variate Analysis between {var_x} and {var_y}")
    display_html(content="")

    fig, axes = plt.subplots(1, 2, figsize=figsize)

    # scatter plot
    sns.scatterplot(
        data,
        x=var_x,
        y=var_y,
        ax=axes[0],
        edgecolors="black",
        **scatter_kwargs
    )
    axes[0].set(title="Scatter Plot")

    # hexbin plot
    col_x = data.loc[:, var_x]
    col_y = data.loc[:, var_y]
    hexbin = axes[1].hexbin(
        x=col_x,
        y=col_y,
        **hexbin_kwargs
    )
    axes[1].set(
        title="Hexbin Plot",
        xlabel=var_x,
        xlim=(col_x.min(), col_x.max()),
        ylim=(col_y.min(), col_y.max())
    )
    cb = plt.colorbar(
        hexbin,
        label="Count"
    )

    plt.tight_layout()
    plt.show()

### **Multivariate plots between 2 numeric and 2 categorical variables**


In [67]:
def  num_cat_facetgrid(data_frame, facet_column, hue_column, x_axis_column, y_axis_column,
    columns_per_row=None, facet_height=3, facet_aspect_ratio=1.2 ):
    """
    Creates a FacetGrid of scatter plots to visualize the relationship between numeric and categorical variables.

    Parameters:
    - data_frame (pd.DataFrame): The DataFrame containing the data.
    - facet_column (str): The name of the categorical column used to create facets (columns) in the grid.
    - hue_column (str): The name of the categorical column used to color the points in the scatter plot.
    - x_axis_column (str): The name of the numeric column to be plotted on the x-axis.
    - y_axis_column (str): The name of the numeric column to be plotted on the y-axis.
    - columns_per_row (int, optional): Number of columns in each row of facets. Defaults to auto-calculate based on the number of unique facets.
    - facet_height (float, optional): Height of each facet in inches. Defaults to 3.
    - facet_aspect_ratio (float, optional): Aspect ratio (height/width) of each facet. Defaults to 1.2.

    Returns:
    - None: Displays the FacetGrid of scatter plots.
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy= data_frame.copy()
    # Determine the number of facets and auto-calculate columns_per_row if not provided
    num_facets = df_copy[facet_column].nunique()

    if columns_per_row is None:
        columns_per_row = max(3, num_facets // 2)  # Ensure at least 3 columns per row

    g = sns.FacetGrid(
        df, col=facet_column, hue=hue_column, col_wrap=columns_per_row, sharex=True, sharey=True, height=facet_height, aspect=facet_aspect_ratio
    )
    g.map(sns.scatterplot, x_axis_column, y_axis_column)

    # Add a legend to each facet
    for ax in g.axes.flatten():
        ax.legend(loc='best')  # Adjust the `loc` parameter as needed

    plt.show()
